# Práctica 3.6. Clasificación Multiclase

In [ ]:
import keras
keras.__version__

En este ejemplo vamos a construir una red para clasificar las noticias de Reuters según una clasificación que contiene 46 temas disjuntos. Por ello, estamos ante un problema de **Clasificación Multi-Clase** (concretamente, de etiqueta única, porque cada ejemplo solo puede ser asignado a una de las clases).

El problema analizado es muy similar al que ya hemos visto de opiniones de IMDB, por lo que profundizaremos únicamente en aquellas fases que lo diferencian.

## 1. El dataset de Reuters

Vamos a trabajar sobre el *dataset de Reuters*, un conjunto de noticias cortas clasificadas por temas (tópicos), que publicó la empresa de noticias Reuters en 1986 y que se ha usado ampliamente como ejemplo en problemas de clasificación. Este dataset hace uso de 46 temas distintos, algunos con más representación que otros, pero en el que, al menos, cada tema etiqueta 10 noticias del conjunto de entrenamiento.

Este datset también viene empaquetado en Keras y su carga es directa y similar a las que hemos realizado en los ejemplos anteriores:

In [ ]:
# Descomenta las siguientes líneas y la última de la celda en caso de obtener el error "Object arrays cannot be loaded when allow_pickle=False"
#import numpy as np
#np_load_old = np.load
#np.load = lambda *a, **k: np_load_old(*a, allow_pickle=True, **k)

from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

#np.load = np_load_old


Tenemos 8,982 ejemplos de entrenamiento y 2,246 ejemplos de test:


In [ ]:
len(train_data)

In [ ]:
len(test_data)

Al igual que en el caso de IMDB, los ejemplos son listas de enteros que representan índices de palabras de un diccionario:


In [ ]:
train_data[10]

Que podemos decodificar de forma completamente análoga a como lo hicimos entonces:


In [ ]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# Observa que nuestros índices fueron desplazados por 3
# porque 0, 1 y 2 son índices reservados para "padding", "start of sequence", y "unknown".
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

In [ ]:
decoded_newswire

La etiqueta asociada a una noticia es un entero entre 0 y 45, que representa el índice de un tópico.


In [ ]:
train_labels[10]

## 2. Preparando los datos

Usamos exactamente las mismas funciones para vectorizar los datos y prepararlos como entrada a la red neuronal:

In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

# Nuestros datos de entrenamiento vectorizados
x_train = vectorize_sequences(train_data)
# Nuestros datos de test vectorizados
x_test = vectorize_sequences(test_data)


La diferencia principal ahora es que antes estas etiquetas eran binarias y ahora son números enteros. Podríamos tratarlas como enteros (tensores escalares) o bien hacer una codificación _one-hot_ tal y como hicimos antes, que será la opción que tomemos siguiendo un patrón similar al realizado para las vectorizaciones anteriores (aunque, en este caso, como cada noticia solo puede tener asociado un tema, los vectores tendrán solo un 1 y todo lo demás 0s):


In [ ]:
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

# Nuestros datos de entrenamiento vectorizados
one_hot_train_labels = to_one_hot(train_labels)
# Nuestros datos de test vectorizados
one_hot_test_labels = to_one_hot(test_labels)

Esta tarea es tan común que hay una función de Keras que realiza ya esta labor, y que vimos en el primer ejemplo que estudiamos de MNIST. Su resultado es exactamente el mismo que el de la función anterior:


In [ ]:
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

## 3. Construyendo la red

Como ahora la salida debe tener 46 unidades, no tiene sentido comprimir las capas anteriores a 16 unidades, ya que estaríamos creando un cuello de botella en esa parte de la red, así que trabajaremos con 2 capas densas de 64 unidades, y una capa densa de salida con 46 unidades (una por cada posible tópico) pero de activación `softmax`:

In [ ]:
from keras import models
from keras import layers
from keras.utils.vis_utils import plot_model

red = models.Sequential()
red.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
red.add(layers.Dense(64, activation='relu'))
red.add(layers.Dense(46, activation='softmax'))

plot_model(red, to_file='ReutersModel_plot.png', show_shapes=True, show_layer_names=True)

![](./imgs/ReutersModel_plot.png)

Destaquemos las diferencias que podemos encontrar respedto al caso anterior:

* La red acaba con una capa densa de 46 unidades, de forma que la salida será un vector de tamaño 46, uno por cada posible clase de salida.

* La última capa usa activación `softmax`, lo que significa que la salida se podrá interpretar como una *distribución de probabilidad* sobre las diferentes clases de salida (un vector positivo de tamaño 46 y que suma 1).

Siguiendo un razonamiento similar al caso del IMDB, la función de pérdida adecuada en este caso es `categorical_crossentropy` (similar, pero para el caso no binario):

In [ ]:
red.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## 4. Validando el modelo

Al igual que antes, vamos a separar 1,000 muestras del conjunto de entrenamiento para formar un conjunto de validación que nos servirá para evaluar cómo funciona el modelo a medida que va aprendiendo (y controlar, por ejemplo, el *overfitting*):

In [ ]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

Ahora entrenemos el modelo con 20 epochs, usando el conjunto de validación para tener información adicional de cómo evoluciona:


In [ ]:
entrenamiento = red.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Podemos mostrar las curvas de pérdida y accuracy en los datos de entrenamiento y validación::


In [ ]:
import matplotlib.pyplot as plt

ent_loss = entrenamiento.history['loss']
val_loss = entrenamiento.history['val_loss']

epochs = range(1, len(ent_loss) + 1)

plt.plot(epochs, ent_loss, 'bo', label='Entrenamiento')
plt.plot(epochs, val_loss, 'b', label='Validación')
plt.title('Pérdida en Entrenamiento y Validación')
plt.xlabel('Epochs')
plt.ylabel('Pérdida')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # limpiar figura

ent_acc = entrenamiento.history['acc']
val_acc = entrenamiento.history['val_acc']

plt.plot(epochs, ent_acc, 'bo', label='Entrenamiento')
plt.plot(epochs, val_acc, 'b', label='Validación')
plt.title('Accuracy en Entrenamiento y Validación')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

Que nos muestran que el modelo comienza a sobreajustar a partir de la 9ª epoch, por lo que entrenaremos un modelo limpio solo durante 9 epochs, y mediremos su rendimiento con los datos de test:


In [ ]:
red2 = models.Sequential()
red2.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
red2.add(layers.Dense(64, activation='relu'))
red2.add(layers.Dense(46, activation='softmax'))

red2.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
red2.fit(partial_x_train,
          partial_y_train,
          epochs=8,
          batch_size=512,
          validation_data=(x_val, y_val))
results = red2.evaluate(x_test, one_hot_test_labels)

In [ ]:
results


El modelo obtenido consigue una accuracy de ~78%. Si la clasificación fuera binaria y estuviera balanceada, un clasificador aleatorio alcanzaría un 50%, pero al ser multi-clase en este caso uno aleatorio esperaría no más del 19%, por lo no lo hace nada mal:


In [ ]:
import copy

test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
float(np.sum(np.array(test_labels) == np.array(test_labels_copy))) / len(test_labels)

## 5. Predicciones en nuevos datos

Usando el método `predict` podemos ver cómo nuestro modelo funciona sobre datos no vistos. Por ejemplo, si trabajamos sobre los datos de test:

In [ ]:
predictions = red2.predict(x_test)

Podemos comprobar que cada entrada de `predictions` es un vector de longitud 46:


In [ ]:
predictions[0].shape

Cuyos coeficientes suman 1:


In [ ]:
np.sum(predictions[0])

Y donde el mayor valor indica la predicción del modelo (la mayor probabilidad):


In [ ]:
np.argmax(predictions[0])

## 6. Salidas Categóricas Discretas

Si hubiéramos decidido manejar las etiquetas directamente como valores enteros, entonces hubiera sido conveniente cambiar de función de pérdida de `categorical_crossentropy` a `sparse_categorical_crossentropy`, que está preparada para medir errores cuando las salidas son categóricas pero no continuas (como en el caso de distribuciones de probabilidad):

In [ ]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [ ]:
red2.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

En realidad, es la misma función que antes, pero con transformaciones internas para manejar valores enteros.


## 7. Tamaño de las capas intermedias

Podemos hacer el experimento de ver qué pasaría si trabajáramos con capas intermedias que producen cuellos de botella a la información que se transmite a lo largo de la red. Por ejemplo, si la segunda capa la cambiamos a 4 unidades:

In [ ]:
red3 = models.Sequential()
red3.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
red3.add(layers.Dense(4, activation='relu'))
red3.add(layers.Dense(46, activation='softmax'))

red3.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
red3.fit(partial_x_train,
          partial_y_train,
          epochs=20,
          batch_size=128,
          validation_data=(x_val, y_val))


Podemos observar que el rendimiento del modelo cae al ~66% de accuracy, un 12% de caida respecto a la aproximación anterior, debido al hecho de que ahora la red intenta comprimir toda la información de las 64 neuronas anteriores en 4 neuronas para volver a descomprimirlo en 46 neuronas de salida, y aunque es capaz de mantener mucha información en esas pocas neuronas, pierde mucha estructura de la información original en ese proceso.


## 8. Trabajo Propuesto

* Intenta usar otro tamaño en las capas, ampliando o reduciendo: 32, 128,...
* Intenta usar más o menos cantidad de capas ocultas.

## 9. Conclusiones

Algunas conclusiones que podemos ir apuntando de este ejemplo:

* Si quieres clasificar entre $N$ clases, tu red debe acabar en una capa densa de tamaño $N$.
* Si la clasificación es disjunta, la última capa debe usar activación `softmax`, de esa forma obtendrás una distribución de probabilidad sobre las N clases de salida.
* Casi con toda seguridad tendrás que usar *Categorical crossentropy* como función de pérdida.
* Hay dos formas de manejar las etiquetas en un problema de clasificación multi-clase:
    * Por _Codificación Categórica_ ("one-hot encoding") y usando `categorical_crossentropy` como función de pérdida.
    * Codificando las etiquetas como enteros, y usando `sparse_categorical_crossentropy` como función de pérdida.
* Si el número de categorías es muy alto, evita los cuellos de botella con capas intermedias demasiado pequeñas.